This notebook is based on [this example](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb#scrollTo=Fqo_hMRZiubM)

In [1]:
!pip install faiss-cpu langchain langchain-community langgraph langchain-openai langchain-anthropic -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.

In [2]:
from langchain.tools.retriever import create_retriever_tool

from langgraph.prebuilt import create_react_agent
from google.colab import userdata


In [3]:
!git clone https://github.com/RadchaneepornC/OfficeBuddyPrime_LLMAgentic

Cloning into 'OfficeBuddyPrime_LLMAgentic'...
remote: Enumerating objects: 254, done.
remote: Counting objects: 100% (254/254), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 254 (delta 100), reused 191 (delta 53), pack-reused 0 (from 0)
Receiving objects: 100% (254/254), 25.84 MiB | 26.04 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [4]:
import json
import os


def read_json_file(file_path):

  try:
    with open(file_path, 'r') as f:
      sample_records = json.load(f)

  except FileNotFoundError:
    print(f"Error: File not found at '{file_path}'. Please check the file path.")
  except json.JSONDecodeError:
    print(f"Error: Invalid JSON format in '{file_path}'. Please check the file content.")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")

  return sample_records



In [5]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')

In [6]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     model="gpt-4.1-mini-2025-04-14",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
# )

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    model="claude-3-7-sonnet-20250219",
    temperature=0,
    max_tokens=2500,
    timeout=None,
    max_retries=2,
)


In [7]:
llm.invoke("Hi")

AIMessage(content="Hello! How can I assist you today? Feel free to ask any questions or let me know what you'd like to discuss.", additional_kwargs={}, response_metadata={'id': 'msg_01F1mnUvcFhCVGXVGWENSFMh', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 8, 'output_tokens': 29, 'server_tool_use': None}, 'model_name': 'claude-3-7-sonnet-20250219'}, id='run--05fa99e7-7148-4d34-a4c0-61d7d4a981ac-0', usage_metadata={'input_tokens': 8, 'output_tokens': 29, 'total_tokens': 37, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

In [17]:
from functools import partial

def ask_agent(question, agent_instance):
    return agent_instance.invoke(question)

# Create a partial function with the agent already provided
agent_call = partial(ask_agent, agent_instance=llm)

In [18]:

def get_metric(answer) :

  def get_usage(key: str, message) -> int :
    return message.response_metadata.get("usage", {}).get(key, 0)

  def token_to_usd(input_token : int, output_token : int , input_per_million = 3, output_per_million = 15) :
    input_price = input_per_million / 1000000
    output_price = output_per_million / 1000000
    return (input_token * input_price + output_token * output_price)

  input_token = 0
  output_token = 0
  for ret in answer['messages'] :
    input_token += get_usage("input_tokens", ret)
    output_token += get_usage("output_tokens", ret)

  price = token_to_usd(input_token=input_token, output_token=output_token)

  return { "input_tokens": input_token, "output_tokens": output_token, "cost_usd":price}



In [19]:
test_path = "OfficeBuddyPrime_LLMAgentic/testset/tax-test-set.json"

In [20]:
testset = read_json_file(test_path)['test_cases']

In [21]:
print(testset)

[{'id': 1, 'question': 'ใครบ้างที่มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา?', 'golden_answer': 'ผู้มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา ได้แก่ ผู้ที่มีเงินได้เกิดขึ้นระหว่างปีที่ผ่านมาโดยมีสถานะอย่างหนึ่งอย่างใด ดังนี้ 1) บุคคลธรรมดา 2) ห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล 3) ผู้ถึงแก่ความตายระหว่างปีภาษี 4) กองมรดกที่ยังไม่ได้แบ่ง 5) วิสาหกิจชุมชนตามกฎหมายว่าด้วยการส่งเสริมวิสาหกิจชุมชน เฉพาะที่เป็นห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล'}, {'id': 2, 'question': 'เงินได้ประเภทใดบ้างที่ต้องเสียภาษีเงินได้บุคคลธรรมดา?', 'golden_answer': 'เงินได้พึงประเมินที่ต้องเสียภาษีเงินได้บุคคลธรรมดา แบ่งเป็น 8 ประเภท ดังนี้: 1. เงินได้ประเภทที่ 1: เงินเดือน ค่าจ้าง เบี้ยเลี้ยง โบนัส และเงินอื่นๆ จากการจ้างแรงงาน 2. เงินได้ประเภทที่ 2: เงินได้จากหน้าที่หรือตำแหน่งงาน เช่น ค่าธรรมเนียม ค่านายหน้า 3. เงินได้ประเภทที่ 3: ค่าแห่งกู๊ดวิลล์ ค่าลิขสิทธิ์ เงินปี 4. เงินได้ประเภทที่ 4: ดอกเบี้ย เงินปันผล เงินส่วนแบ่งกำไร ผลประโยชน์จากการโอนหุ้น 5. เงินได้ประเภทที่ 5: เงินได้จากการให้เช่าทรัพย์สิน 

In [24]:
import time
results = []
for items in testset[:] :
  id, question, gold_answer = items.values()
  start = time.time()
  answer = agent_call(question)
  stop = time.time()
  metrics = get_metric({"messages":[answer]})
  metrics['time_seconds'] = stop - start
  results.append({
    "id": id,
    "question": question,
    "inference_result": {"messages":[answer]}["messages"][-1].content,
    "gold_answer": gold_answer,
    "metrics": metrics
  })


In [25]:
json.dumps({
  "description": "Thai Personal Income Tax RAG Test Set for evaluating retrieval and QA performance results from BGE-M3 with Claude 3.7 sonnet",
  "version": "1.0",
  "date_created": "2025-05-08",
  "test_cases": results
})

'{"description": "Thai Personal Income Tax RAG Test Set for evaluating retrieval and QA performance results from BGE-M3 with Claude 3.7 sonnet", "version": "1.0", "date_created": "2025-05-08", "test_cases": [{"id": 1, "question": "\\u0e43\\u0e04\\u0e23\\u0e1a\\u0e49\\u0e32\\u0e07\\u0e17\\u0e35\\u0e48\\u0e21\\u0e35\\u0e2b\\u0e19\\u0e49\\u0e32\\u0e17\\u0e35\\u0e48\\u0e40\\u0e2a\\u0e35\\u0e22\\u0e20\\u0e32\\u0e29\\u0e35\\u0e40\\u0e07\\u0e34\\u0e19\\u0e44\\u0e14\\u0e49\\u0e1a\\u0e38\\u0e04\\u0e04\\u0e25\\u0e18\\u0e23\\u0e23\\u0e21\\u0e14\\u0e32?", "inference_result": "\\u0e1a\\u0e38\\u0e04\\u0e04\\u0e25\\u0e17\\u0e35\\u0e48\\u0e21\\u0e35\\u0e2b\\u0e19\\u0e49\\u0e32\\u0e17\\u0e35\\u0e48\\u0e40\\u0e2a\\u0e35\\u0e22\\u0e20\\u0e32\\u0e29\\u0e35\\u0e40\\u0e07\\u0e34\\u0e19\\u0e44\\u0e14\\u0e49\\u0e1a\\u0e38\\u0e04\\u0e04\\u0e25\\u0e18\\u0e23\\u0e23\\u0e21\\u0e14\\u0e32 \\u0e44\\u0e14\\u0e49\\u0e41\\u0e01\\u0e48:\\n\\n1. \\u0e1a\\u0e38\\u0e04\\u0e04\\u0e25\\u0e18\\u0e23\\u0e23\\u0e21\\u0e14\\u0e

In [26]:


import json

output_data = {
  "description": "Thai Personal Income Tax RAG Test Set for evaluating retrieval and QA performance results from BGE-M3 with Claude 3.7 sonnet",
  "version": "1.0",
  "date_created": "2025-05-08",
  "test_cases": results
}

# Specify the desired file path
output_file_path = "evaluation_results.json"

try:
  with open(output_file_path, 'w') as f:
    json.dump(output_data, f, indent=4)  # Use indent for pretty printing
  print(f"Data successfully saved to {output_file_path}")

except Exception as e:
  print(f"An error occurred while saving the file: {e}")


Data successfully saved to evaluation_results.json
